# 19.5.4 Random Forest Vs. Deep Learning Model

## skipped skill drill

## Import our dependencies

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import tensorflow as tf

## Import our input dataset

In [2]:
loans_df = pd.read_csv('./Resources/loan_status.csv')
loans_df.head()

,Loan_Status,Current_Loan_Amount,Term,Credit_Score,Annual_Income,Years_in_current_job,Home_Ownership,Purpose,Monthly_Debt,Years_of_Credit_History,Months_since_last_delinquent,Number_of_Open_Accounts,Number_of_Credit_Problems,Current_Credit_Balance,Maximum_Open_Credit,Bankruptcies,Tax_Liens
0,Fully_Paid,99999999,Short_Term,741.0,2231892.0,8_years,Own_Home,Debt_Consolidation,29200.53,14.9,29.0,18,1,297996,750090.0,0.0,0.0
1,Fully_Paid,217646,Short_Term,730.0,1184194.0,<_1_year,Home_Mortgage,Debt_Consolidation,10855.08,19.6,10.0,13,1,122170,272052.0,1.0,0.0
2,Fully_Paid,548746,Short_Term,678.0,2559110.0,2_years,Rent,Debt_Consolidation,18660.28,22.6,33.0,4,0,437171,555038.0,0.0,0.0
3,Fully_Paid,99999999,Short_Term,728.0,714628.0,3_years,Rent,Debt_Consolidation,11851.06,16.0,76.0,16,0,203965,289784.0,0.0,0.0
4,Fully_Paid,99999999,Short_Term,740.0,776188.0,<_1_year,Own_Home,Debt_Consolidation,11578.22,8.5,25.0,6,0,134083,220220.0,0.0,0.0


In [3]:
# Generate our categorical variable list

In [4]:
loans_cat = loans_df.dtypes[loans_df.dtypes == "object"].index.tolist()

## Check the number of unique values in each column

In [5]:
loans_df[loans_cat].nunique()

Loan_Status              2
Term                     2
Years_in_current_job    11
Home_Ownership           4
Purpose                  7
dtype: int64

## Check the unique value counts to see if binning is required

In [6]:
loans_df.Years_in_current_job.value_counts()

10+_years    13149
2_years       3225
3_years       2997
<_1_year      2699
5_years       2487
4_years       2286
1_year        2247
6_years       2109
7_years       2082
8_years       1675
9_years       1467
Name: Years_in_current_job, dtype: int64

## Create a OneHotEncoder instance

In [7]:
enc = OneHotEncoder(sparse=False)

## Fit and transform the OneHotEncoder using the categorical variable list

In [8]:
encode_df = pd.DataFrame(enc.fit_transform(loans_df[loans_cat]))

## Add the encoded variable names to the DataFrame

In [9]:
encode_df.columns = enc.get_feature_names(loans_cat)
encode_df.head()

C:\ProgramData\Anaconda3\envs\Mlenv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,Loan_Status_Fully_Paid,Loan_Status_Not_Paid,Term_Long_Term,Term_Short_Term,Years_in_current_job_10+_years,Years_in_current_job_1_year,Years_in_current_job_2_years,Years_in_current_job_3_years,Years_in_current_job_4_years,Years_in_current_job_5_years,...,Home_Ownership_Home_Mortgage,Home_Ownership_Own_Home,Home_Ownership_Rent,Purpose_Business_Loan,Purpose_Buy_House,Purpose_Buy_a_Car,Purpose_Debt_Consolidation,Purpose_Home_Improvements,Purpose_Medical_Bills,Purpose_Other
0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


## Merge one-hot encoded features and drop the originals

In [10]:
loans_df = loans_df.merge(encode_df,left_index=True, right_index=True)
loans_df = loans_df.drop(loans_cat,1)
loans_df.head()

C:\ProgramData\Anaconda3\envs\Mlenv\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


,Current_Loan_Amount,Credit_Score,Annual_Income,Monthly_Debt,Years_of_Credit_History,Months_since_last_delinquent,Number_of_Open_Accounts,Number_of_Credit_Problems,Current_Credit_Balance,Maximum_Open_Credit,...,Home_Ownership_Home_Mortgage,Home_Ownership_Own_Home,Home_Ownership_Rent,Purpose_Business_Loan,Purpose_Buy_House,Purpose_Buy_a_Car,Purpose_Debt_Consolidation,Purpose_Home_Improvements,Purpose_Medical_Bills,Purpose_Other
0,99999999,741.0,2231892.0,29200.53,14.9,29.0,18,1,297996,750090.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,217646,730.0,1184194.0,10855.08,19.6,10.0,13,1,122170,272052.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,548746,678.0,2559110.0,18660.28,22.6,33.0,4,0,437171,555038.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,99999999,728.0,714628.0,11851.06,16.0,76.0,16,0,203965,289784.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,99999999,740.0,776188.0,11578.22,8.5,25.0,6,0,134083,220220.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


## Remove loan status target from features data

In [11]:
y = loans_df.Loan_Status_Fully_Paid
X = loans_df.drop(columns=["Loan_Status_Fully_Paid","Loan_Status_Not_Paid"])

# Split training/test datasets

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

## Create a StandardScaler instance

In [13]:
scaler = StandardScaler()

## Fit the StandardScaler

In [14]:
X_scaler = scaler.fit(X_train)

## Scale the data

In [15]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Create a random forest classifier.

In [16]:
rf_model = RandomForestClassifier(n_estimators=128, random_state=78)

## Fitting the model

In [17]:
rf_model = rf_model.fit(X_train_scaled, y_train)

## Evaluate the model

In [18]:
y_pred = rf_model.predict(X_test_scaled)
print(f" Random forest predictive accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Random forest predictive accuracy: 0.849


In [19]:
# Define the model - deep neural net

In [20]:
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  24
hidden_nodes_layer2 = 12

nn = tf.keras.models.Sequential()


## First hidden layer

In [21]:
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer

In [22]:
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

## Output layer

In [23]:
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

## Compile the Sequential model together and customize metrics

In [24]:
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

## Train the model

In [25]:
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
854/854 [==============================] - 1s 1ms/step - loss: 0.4432 - accuracy: 0.8189
Epoch 2/50
854/854 [==============================] - 1s 2ms/step - loss: 0.3862 - accuracy: 0.8493
Epoch 3/50
854/854 [==============================] - 1s 2ms/step - loss: 0.3820 - accuracy: 0.8493
Epoch 4/50
854/854 [==============================] - 1s 2ms/step - loss: 0.3798 - accuracy: 0.8495
Epoch 5/50
854/854 [==============================] - 1s 1ms/step - loss: 0.3780 - accuracy: 0.8496
Epoch 6/50
854/854 [==============================] - 1s 1ms/step - loss: 0.3774 - accuracy: 0.8493
Epoch 7/50
854/854 [==============================] - 1s 1ms/step - loss: 0.3760 - accuracy: 0.8494
Epoch 8/50
854/854 [==============================] - 1s 1ms/step - loss: 0.3754 - accuracy: 0.8495
Epoch 9/50
854/854 [==============================] - 1s 1ms/step - loss: 0.3744 - accuracy: 0.8497
Epoch 10/50
854/854 [==============================] - 1s 1ms/step - loss: 0.3735 - accuracy: 0.8500

## Evaluate the model using the test data

In [26]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

285/285 - 0s - loss: 0.3920 - accuracy: 0.8474 - 365ms/epoch - 1ms/step
Loss: 0.3920218348503113, Accuracy: 0.8473533987998962


# 19.6.1 Checkpoints Are Not Just for Video Games (maybe needed to open a new notebook?, last line of code isn't working)

In [27]:
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 24)                888       
                                                                 
 dense_1 (Dense)             (None, 12)                300       
                                                                 
 dense_2 (Dense)             (None, 1)                 13        
                                                                 
Total params: 1,201
Trainable params: 1,201
Non-trainable params: 0
_________________________________________________________________


## Import checkpoint dependencies

In [28]:
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames

In [29]:
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Compile the model

In [30]:
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every 5 epochs

In [31]:
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=1000)

# Train the model

In [32]:
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

Epoch 1/100
854/854 [==============================] - 2s 2ms/step - loss: 0.3603 - accuracy: 0.8527
Epoch 2/100
138/854 [===>..........................] - ETA: 1s - loss: 0.3641 - accuracy: 0.8469
Epoch 2: saving model to checkpoints\weights.02.hdf5
854/854 [==============================] - 1s 2ms/step - loss: 0.3596 - accuracy: 0.8531
Epoch 3/100
273/854 [========>.....................] - ETA: 0s - loss: 0.3597 - accuracy: 0.8529
Epoch 3: saving model to checkpoints\weights.03.hdf5
854/854 [==============================] - 1s 2ms/step - loss: 0.3593 - accuracy: 0.8532
Epoch 4/100
429/854 [==============>...............] - ETA: 0s - loss: 0.3557 - accuracy: 0.8544
Epoch 4: saving model to checkpoints\weights.04.hdf5
854/854 [==============================] - 1s 1ms/step - loss: 0.3594 - accuracy: 0.8528
Epoch 5/100
578/854 [===================>..........] - ETA: 0s - loss: 0.3572 - accuracy: 0.8524
Epoch 5: saving model to checkpoints\weights.05.hdf5
854/854 [=======================

## Evaluate the model using the test dat

In [33]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

285/285 - 0s - loss: 0.4002 - accuracy: 0.8436 - 458ms/epoch - 2ms/step
Loss: 0.4002450108528137, Accuracy: 0.8436195850372314


## Define the model - deep neural net

In [39]:
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn_new = tf.keras.models.Sequential()

## First hidden layer

In [40]:
nn_new.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

## Second hidden layer

In [41]:
nn_new.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

## Output layer

In [42]:
nn_new.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

## Compile the model

In [43]:
nn_new.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

## Restore the model weights

In [45]:
nn_new.load_weights("checkpoints/weights.100.hdf5")

ValueError: Cannot assign value to variable ' dense_3/kernel:0': Shape mismatch.The variable shape (36, 8), and the assigned value shape (36, 24) are incompatible.

## Evaluate the model using the test data

In [46]:
model_loss, model_accuracy = nn_new.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

285/285 - 0s - loss: 0.5898 - accuracy: 0.7339 - 408ms/epoch - 1ms/step
Loss: 0.5898188352584839, Accuracy: 0.7339116930961609


# 19.6.2 For Best Results, Please Save After Training

## Export our model to HDF5 file

In [47]:
nn_new.save("trained_attrition.h5")

## Import the model to a new object

In [48]:
nn_imported = tf.keras.models.load_model('trained_attrition.h5')

## Evaluate the model using the test data

In [49]:
model_loss, model_accuracy = nn_new.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

285/285 - 0s - loss: 0.5898 - accuracy: 0.7339 - 312ms/epoch - 1ms/step
Loss: 0.5898188352584839, Accuracy: 0.7339116930961609
